In [ ]:
import marimo as mo

# Week 9: Convolutional Neural Networks (CNNs)**IME775: Data Driven Modeling and Optimization**📖 **Reference**: Krishnendu Chaudhury. *Math and Architectures of Deep Learning*, Chapter 9---## Learning Objectives- Understand the convolution operation mathematically- Master CNN building blocks: convolution, pooling, padding- Learn classic architectures- Visualize what CNNs learn

In [ ]:
import numpy as npimport matplotlib.pyplot as pltfrom scipy import signal

## 9.1 Why Convolutions?**Problems with Fully Connected layers for images:**- Too many parameters (224×224×3 image → 150M params for first layer!)- No spatial structure- No translation invariance**Convolution advantages:**- Parameter sharing (same filter everywhere)- Local connectivity- Translation equivariance

## 9.2 The Convolution OperationFor 2D input $X$ and kernel $W$ of size $k \times k$:$$(X * W)[i,j] = \sum_{m=0}^{k-1} \sum_{n=0}^{k-1} X[i+m, j+n] \cdot W[m,n]$$**Sliding the filter across the image and computing dot products.**

In [ ]:
# Visualize convolution step by step    k = W.shape[0]    if padding > 0:        X_padded = np.pad(X, padding, mode='constant')    else:        X_padded = X    fig, axes = plt.subplots(1, 4, figsize=(16, 4))    # Input with highlight    ax1 = axes[0]    ax1.imshow(X_padded, cmap='gray', vmin=0, vmax=1)    # Highlight the patch    rect = plt.Rectangle((step_j - 0.5, step_i - 0.5), k, k,                          fill=False, edgecolor='red', linewidth=3)    ax1.add_patch(rect)    ax1.set_title(f'Input (patch at [{step_i},{step_j}])')    ax1.axis('off')    # Extracted patch    ax2 = axes[1]    patch = X_padded[step_i:step_i+k, step_j:step_j+k]    ax2.imshow(patch, cmap='gray', vmin=0, vmax=1)    for i in range(k):        for j in range(k):            ax2.text(j, i, f'{patch[i,j]:.1f}', ha='center', va='center',                     fontsize=12, color='red' if patch[i,j] > 0.5 else 'blue')    ax2.set_title('Extracted Patch')    ax2.axis('off')    # Filter    ax3 = axes[2]    ax3.imshow(W, cmap='RdBu', vmin=-1, vmax=1)    for i in range(k):        for j in range(k):            ax3.text(j, i, f'{W[i,j]:.1f}', ha='center', va='center', fontsize=12)    ax3.set_title('Filter (Kernel)')    ax3.axis('off')    # Element-wise product    ax4 = axes[3]    product = patch * W    ax4.imshow(product, cmap='RdBu', vmin=-1, vmax=1)    for i in range(k):        for j in range(k):            ax4.text(j, i, f'{product[i,j]:.2f}', ha='center', va='center', fontsize=10)    ax4.set_title(f'Element-wise Product\nSum = {product.sum():.2f}')    ax4.axis('off')    plt.tight_layout()    return fig# Create exampleX_demo = np.array([    [0, 0, 0, 0, 0, 0],    [0, 1, 1, 1, 0, 0],    [0, 1, 1, 1, 0, 0],    [0, 1, 1, 1, 0, 0],    [0, 0, 0, 0, 0, 0],    [0, 0, 0, 0, 0, 0],], dtype=float)W_demo = np.array([    [1, 0, -1],    [1, 0, -1],    [1, 0, -1],], dtype=float) / 3fig2 = visualize_conv_step(X_demo, W_demo, step_i=1, step_j=0, padding=0)fig2

## 9.3 Stride and Padding**Stride**: Step size when sliding the filter$$\text{Output size} = \lfloor \frac{n - k}{\text{stride}} \rfloor + 1$$**Padding**: Add zeros around input$$\text{Output size} = \lfloor \frac{n + 2p - k}{\text{stride}} \rfloor + 1$$

## 9.4 Pooling Layers**Purpose**: Reduce spatial dimensions, add translation invariance| Type | Operation | Use ||------|-----------|-----|| Max | Take maximum | Most common || Average | Take mean | Some architectures || Global Avg | Mean over all spatial | Before classifier |

## 9.5 CNN Architecture Evolution| Year | Architecture | Key Innovation ||------|--------------|----------------|| 1998 | LeNet-5 | First successful CNN || 2012 | AlexNet | ReLU, Dropout, GPU || 2014 | VGGNet | Small 3×3 filters || 2015 | ResNet | Skip connections || 2017 | MobileNet | Depthwise separable |

In [ ]:
# Visualize feature hierarchyfig5, axes5 = plt.subplots(1, 4, figsize=(16, 4))# Simulate different layer featuresnp.random.seed(42)# Layer 1: Edgesedge_filters = np.array([    [[1, 0, -1], [2, 0, -2], [1, 0, -1]],  # Vertical    [[1, 2, 1], [0, 0, 0], [-1, -2, -1]],  # Horizontal    [[0, 1, 2], [-1, 0, 1], [-2, -1, 0]],  # Diagonal])for i, filt in enumerate(edge_filters[:3]):    ax = axes5[0] if i == 0 else Noneaxes5[0].set_title('Layer 1: Edges', fontsize=12)for i, filt in enumerate(edge_filters):    y_offset = i * 0.35    for ii in range(3):        for jj in range(3):            color = 'red' if filt[ii, jj] > 0 else ('blue' if filt[ii, jj] < 0 else 'gray')            axes5[0].add_patch(plt.Rectangle((jj/4 + 0.1, 0.55 - y_offset - ii/4),                                               0.2, 0.2, facecolor=color, alpha=0.5))axes5[0].set_xlim(0, 1)axes5[0].set_ylim(0, 1)axes5[0].axis('off')# Layer 2-3: Textures/Partsaxes5[1].text(0.5, 0.7, '🔲 Corners', fontsize=20, ha='center')axes5[1].text(0.5, 0.5, '〰️ Curves', fontsize=20, ha='center')axes5[1].text(0.5, 0.3, '⬛ Patterns', fontsize=20, ha='center')axes5[1].set_title('Layer 2-3: Textures & Parts', fontsize=12)axes5[1].axis('off')# Layer 4: Partsaxes5[2].text(0.5, 0.7, '👁️ Eyes', fontsize=20, ha='center')axes5[2].text(0.5, 0.5, '👃 Noses', fontsize=20, ha='center')axes5[2].text(0.5, 0.3, '🚗 Wheels', fontsize=20, ha='center')axes5[2].set_title('Layer 4: Object Parts', fontsize=12)axes5[2].axis('off')# Layer 5: Objectsaxes5[3].text(0.5, 0.7, '😺 Cats', fontsize=20, ha='center')axes5[3].text(0.5, 0.5, '🚙 Cars', fontsize=20, ha='center')axes5[3].text(0.5, 0.3, '🏠 Houses', fontsize=20, ha='center')axes5[3].set_title('Layer 5: Full Objects', fontsize=12)axes5[3].axis('off')plt.suptitle('CNN Feature Hierarchy: Simple → Complex', fontsize=14, y=1.05)plt.tight_layout()fig5

## 9.6 Receptive FieldThe region of input that affects one output neuron.$$R = 1 + \sum_{l=1}^{L} (k_l - 1) \prod_{i=1}^{l-1} s_i$$

## Summary| Component | Purpose ||-----------|---------|| **Convolution** | Extract local features, parameter sharing || **Stride** | Control downsampling || **Padding** | Control output size || **Pooling** | Reduce dimensions, translation invariance || **Depth** | Learn hierarchical features |---## References- **Primary**: Krishnendu Chaudhury. *Math and Architectures of Deep Learning*, Chapter 9.- **LeNet**: LeCun et al. (1998)- **AlexNet**: Krizhevsky et al. (2012)- **VGG**: Simonyan & Zisserman (2015)## Connection to ML Refined CurriculumCNNs automate the feature engineering from Week 9:- Manual feature extraction → Learned convolutional filters- Hierarchical representation learning